In [1]:
import numpy as np
from wildcat.util.matrix import random_symmetric_matrix
from wildcat.solver.qubo_solver import QuboSolver

In [2]:
problem_str = '''
1? ??
?? 2?

?3 ??
?? ?4'''

# Indices of array are:
#  0  1 |  2  3
#  4  5 |  6  7
# ------+-------
#  8  9 | 10 11
# 12 13 | 14 15

In [3]:
# Remove whitespace and newline
import re
problem_str = re.sub(r'[^0-9?]', '', problem_str)

In [4]:
problem_str

'1?????2??3?????4'

In [5]:
n = 4**3
grouping = [
    # rows
    [ 0,  1,  2,  3],
    [ 4,  5,  6,  7],
    [ 8,  9, 10, 11],
    [12, 13, 14, 15],
    # columns
    [ 0,  4,  8, 12],
    [ 1,  5,  9, 13],
    [ 2,  6, 10, 14],
    [ 3,  7, 11, 15],
    # blocks
    [ 0,  1,  4,  5],
    [ 2,  3,  6,  7],
    [ 8,  9, 12, 13],
    [10, 11, 14, 15],
]

In [675]:
#import random

# Bits arrangement:
# i = 0..15: i-th value is 1
# i = 16..31: (i-16)-th value is 2
# i = 32..47: (i-32)-th value is 3
# i = 48..63: (i-48)-th value is 4

qubo = np.eye(n) * -1

for g in grouping:
    for i in range(4):
        for j in range(3):
            for k in range(j+1, 4):
                qubo[i*16 + g[j], i*16 + g[k]] += 2

for i in range(16):
    for j in range(3):
        for k in range(j+1, k):
            qubo[i+j*16, i+k*16] += 2.5

for i, v in enumerate(problem_str):
    if v == '?':
        continue
    v = int(v) - 1
    qubo[v*16 + i, v*16 + i] -= 20
    for j in range(4):
        qubo[j*16 + i, j*16 + i] += 4
    for g in grouping:
        if i not in g:
            continue
        for j in g:
            qubo[v*16 + j, v*16 + j] += 3 + v*0.3

In [715]:
solver = QuboSolver(qubo)

foo = None
def callback(arrangement):
    global foo
    foo = arrangement
    print(sum(foo>0))
solver.solve(callback).result()

16


<Response [200]>

In [716]:
board = ['?'] * 16
for i,b in enumerate(foo.tolist()):
    if b == 1:
        if board[i%16] != '?':
            print('!', end='')
        board[i%16] = (i//16) + 1

In [717]:
print('''{}{} {}{}
{}{} {}{}

{}{} {}{}
{}{} {}{}'''.format(*board))

12 43
34 21

43 12
21 34
